In [1]:
# Importing important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import seed
from random import random
from math import exp
from csv import reader

In [2]:
# https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/
# STEP 1
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [3]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [4]:
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [5]:
# STEP 2
# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [6]:
# STEP 3
# Backpropagate error (loss function) and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(neuron['output'] - expected[j])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [7]:
# STEP 4
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

In [8]:
# STEP 5
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] -= l_rate * neuron['delta']

In [9]:
# STEP 6
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)

In [10]:
# STEP 7 
# Iterate and Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error/1000))

In [11]:
# Initializing, Training and testing the network using forward and backward propagation

data = [[2.7810836,2.550537003,0  ],
           [1.465489372,2.362125076,0],
           [3.396561688,4.400293529,0],
           [1.38807019,1.850220317,0 ],
           [3.06407232,3.005305973,0 ],
           [7.627531214,2.759262235,1],
           [5.332441248,2.088626775,1],
           [6.922596716,1.77106367,1 ],
           [8.675418651,-0.24206865,1],
           [7.673756466,3.508563011,1]]
print(data)
    
n_inputs = len(data[0]) - 1
n_outputs = len(set([row[-1] for row in data]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, data, 0.5, 20, n_outputs)
for layer in network:
    print(layer)

[[2.7810836, 2.550537003, 0], [1.465489372, 2.362125076, 0], [3.396561688, 4.400293529, 0], [1.38807019, 1.850220317, 0], [3.06407232, 3.005305973, 0], [7.627531214, 2.759262235, 1], [5.332441248, 2.088626775, 1], [6.922596716, 1.77106367, 1], [8.675418651, -0.24206865, 1], [7.673756466, 3.508563011, 1]]
>epoch=0, lrate=0.500, error=0.007
>epoch=1, lrate=0.500, error=0.006
>epoch=2, lrate=0.500, error=0.006
>epoch=3, lrate=0.500, error=0.005
>epoch=4, lrate=0.500, error=0.005
>epoch=5, lrate=0.500, error=0.005
>epoch=6, lrate=0.500, error=0.005
>epoch=7, lrate=0.500, error=0.005
>epoch=8, lrate=0.500, error=0.005
>epoch=9, lrate=0.500, error=0.004
>epoch=10, lrate=0.500, error=0.004
>epoch=11, lrate=0.500, error=0.004
>epoch=12, lrate=0.500, error=0.003
>epoch=13, lrate=0.500, error=0.003
>epoch=14, lrate=0.500, error=0.003
>epoch=15, lrate=0.500, error=0.003
>epoch=16, lrate=0.500, error=0.002
>epoch=17, lrate=0.500, error=0.002
>epoch=18, lrate=0.500, error=0.002
>epoch=19, lrate=0.5

In [12]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

count = 0
# Using neural network to make prediction
for row in data:
    prediction = predict(network, row)
    if prediction == row[-1]:
        count += 1
    print('Expected=%d, Predicted=%d' % (row[-1], prediction))
    
accuracy = count/len(data)
print('Accuracy of the trained model: ', accuracy*100, '%')

Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Accuracy of the trained model:  100.0 %


In [13]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())


# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup
 
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [14]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [15]:
from random import randrange

# Test Backprop on Seeds dataset
seed(1)

# load and prepare data
filename = 'seeds_dataset.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
    
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# Evaluate the model
n_folds = 5
l_rate = 0.3
n_epoch = 50
n_hidden = 5
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

# print('Scores: %s' % scores)
print('Accuracy of the Trained Model: %.3f%%' % (sum(scores)/float(len(scores))))

>epoch=0, lrate=0.300, error=0.133
>epoch=1, lrate=0.300, error=0.111
>epoch=2, lrate=0.300, error=0.103
>epoch=3, lrate=0.300, error=0.089
>epoch=4, lrate=0.300, error=0.075
>epoch=5, lrate=0.300, error=0.067
>epoch=6, lrate=0.300, error=0.061
>epoch=7, lrate=0.300, error=0.056
>epoch=8, lrate=0.300, error=0.053
>epoch=9, lrate=0.300, error=0.049
>epoch=10, lrate=0.300, error=0.045
>epoch=11, lrate=0.300, error=0.042
>epoch=12, lrate=0.300, error=0.039
>epoch=13, lrate=0.300, error=0.036
>epoch=14, lrate=0.300, error=0.034
>epoch=15, lrate=0.300, error=0.032
>epoch=16, lrate=0.300, error=0.030
>epoch=17, lrate=0.300, error=0.029
>epoch=18, lrate=0.300, error=0.028
>epoch=19, lrate=0.300, error=0.027
>epoch=20, lrate=0.300, error=0.026
>epoch=21, lrate=0.300, error=0.025
>epoch=22, lrate=0.300, error=0.025
>epoch=23, lrate=0.300, error=0.024
>epoch=24, lrate=0.300, error=0.023
>epoch=25, lrate=0.300, error=0.023
>epoch=26, lrate=0.300, error=0.023
>epoch=27, lrate=0.300, error=0.022
>e

>epoch=41, lrate=0.300, error=0.019
>epoch=42, lrate=0.300, error=0.019
>epoch=43, lrate=0.300, error=0.019
>epoch=44, lrate=0.300, error=0.019
>epoch=45, lrate=0.300, error=0.018
>epoch=46, lrate=0.300, error=0.018
>epoch=47, lrate=0.300, error=0.018
>epoch=48, lrate=0.300, error=0.018
>epoch=49, lrate=0.300, error=0.018
Accuracy of the Trained Model: 93.333%
